<a href="https://colab.research.google.com/github/aayushme/Aayush_AI_Squad_Project1/blob/master/Musical_Instrumets_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Kaggle Database***

**Calling Required Libraries**

In [3]:
import os
import pandas as pd

**Importing Kaggle.json**

In [4]:
from google.colab import files
my_file=files.upload()

Saving kaggle.json to kaggle (1).json


**Kaggle Cred**


In [5]:
cred = pd.read_json("kaggle.json",lines=True)
os.environ['KAGGLE_USERNAME'] = cred.iloc[0][0]
os.environ['KAGGLE_KEY'] = cred.iloc[0][1]

**Downloading Database**

In [6]:
!kaggle datasets download -d aayushme/music-instruments-classification

music-instruments-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip "*.zip"

In [8]:
os.listdir("instruments_data")

['saxophone',
 'piano',
 'bass_guitar',
 'gramophone',
 'flute',
 'acoustic_guitar',
 'drums',
 'harp',
 'tabla',
 'violen']

#**Data Augmentation**

**Importing Libraries**

In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *
from fastai.metrics import error_rate
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
img_counter=[]
img_counter.append(10)

**Callling Transform Function and creating data class**

In [10]:
path=""
tfms = get_transforms(do_flip=True)
data = ImageDataBunch.from_folder( Path("instruments_data"), valid_pct=0.2, bs=64, size=224, ds_tfms=get_transforms())
data.classes

['acoustic_guitar',
 'bass_guitar',
 'drums',
 'flute',
 'gramophone',
 'harp',
 'piano',
 'saxophone',
 'tabla',
 'violen']

In [19]:
for c in enumerate(data.classes):
    images = os.listdir("instruments_data/"+str(c[1]))
    print(c[1])
    print(len(images))

acoustic_guitar
1904
bass_guitar
1274
drums
1509
flute
750
gramophone
1832
harp
737
piano
223
saxophone
374
tabla
336
violen
342


**`Start Applying Augmentation`**

**Dihedral Function**

In [20]:
def apply_dihedral(image, folder, name):
  img_counter = folder + '/' + 'dihedral' + str(name.split('.')[0])
  idx = 0
  for i in range(1,5,3):
    new_img = dihedral(image, i)
    dest = str(img_counter)+ str(idx) +".jpg"
    new_img.save(dest)
    idx += 1



**Brightness Function**

In [21]:
def change_brightness(image, folder, name):
  img_counter = folder + '/' + 'brightness' + str(name.split('.')[0])
  idx = 0
  for change in np.linspace(0.2,0.8,2):
    new_img = brightness(image, change)
    dest = str(img_counter)+ str(idx) +".jpg"
    new_img.save(dest)
    idx += 1

In [22]:

for c in enumerate(data.classes):
  images = os.listdir("instruments_data/"+str(c[1]))

  for img in images:
    try:
      change_brightness(open_image("instruments_data/"+str(c[1])+"/"+img),"instruments_data/"+str(c[1]) , img)

    except:
      print("Not Done Brightness Changes")

    try:
      apply_dihedral(open_image("instruments_data/"+str(c[1])+"/"+img),"instruments_data/"+str(c[1]) , img)

    except:
      print("Not Done Dihedral Changes")



**Zoom Crop all the images of the dataset**

> We are creating four images form each image of dataset and saving them

> We are using zoom cropping property


#**Training Model**

**Calling Resnet50 into learn variable**

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=accuracy)

**Training the data**

In [ ]:
learn.fit_one_cycle(4)

**Plotting The Learning Rate**

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

**Mounting Drive**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Making Directory in drive and saving path in `my_dir` variable**

In [ ]:
try: 
       
    if not os.path.exists('/content/drive/My Drive/Aayush_Project/Music_Instruments'): 
        os.makedirs('/content/drive/My Drive/Aayush_Project/Music_Instruments') 
  
except OSError: 
    print ('Error: Creating directory of data') 


In [ ]:
my_dir="/content/drive/My Drive/Aayush_Project/Music_Instruments"

**Saving The state with name `Stage-1`**

In [ ]:
learn.save(my_dir+"/Stage-1")

**Export stage-1 to drive**

In [ ]:
learn.export(my_dir+"/export.pkl")